In [3]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/train_preprocessed.csv')

In [23]:
val_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/val_preprocessed.csv')

In [14]:
train_X = train_df['preprocessed_text'].to_list()
train_Y = (train_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [24]:
val_X = val_df['preprocessed_text'].to_list()
val_Y = (val_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [16]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_X)

In [17]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [22]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [25]:
history = model.fit(x = train_X, y = train_Y, epochs=10,
                    validation_data=(val_X, val_Y),
                    validation_steps=100)

Epoch 1/10
331/331 [==============================] - ETA: 0s - loss: 0.6392 - accuracy: 0.6710

331/331 [==============================] - 44s 105ms/step - loss: 0.6392 - accuracy: 0.6710 - val_loss: 0.6236 - val_accuracy: 0.6545
Epoch 2/10
331/331 [==============================] - 29s 86ms/step - loss: 0.5672 - accuracy: 0.6715
Epoch 3/10
331/331 [==============================] - 26s 77ms/step - loss: 0.4268 - accuracy: 0.7881
Epoch 4/10
331/331 [==============================] - 27s 81ms/step - loss: 0.3176 - accuracy: 0.8650
Epoch 5/10
331/331 [==============================] - 26s 77ms/step - loss: 0.2530 - accuracy: 0.8963
Epoch 6/10
331/331 [==============================] - 26s 77ms/step - loss: 0.2080 - accuracy: 0.9188
Epoch 7/10
331/331 [==============================] - 25s 76ms/step - loss: 0.1722 - accuracy: 0.9366
Epoch 8/10
331/331 [==============================] - 26s 78ms/step - loss: 0.1472 - accuracy: 0.9462
Epoch 9/10
331/331 [==============================] - 26s 77ms/step - loss: 0.1289 - accuracy: 0.9546
Epoch 10/10
331/331 [=============================

In [39]:
val_pred = model.predict(val_X)

83/83 [==============================] - 1s 17ms/step


In [40]:
val_loss, val_acc = model.evaluate(val_X, val_Y)

print('val Loss:', val_loss)
print('val Accuracy:', val_acc)

83/83 [==============================] - 1s 17ms/step - loss: 1.2223 - accuracy: 0.7262
val Loss: 1.2222943305969238
val Accuracy: 0.7262084484100342


In [41]:
val_pred = np.where(val_pred > 0.5, 1, 0)

In [42]:
from sklearn.metrics import f1_score

f1_score(val_Y, val_pred, average='micro')

0.7262084592145015